In [1]:
import pandas as pd
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.feature_selection import f_classif
from sklearn.feature_selection import VarianceThreshold
import numpy as np

In [2]:
df = pd.read_csv("30.csv")

In [2]:
df = pd.read_csv("res.csv")
inputs = df.iloc[:,3:].values
outs = df["labels"].values



selector = SelectKBest(f_classif, k=5)
X_new = selector.fit_transform(inputs, outs)
cols = selector.get_support(indices=True)
selected_patterns = np.take(df.columns.values,[x+3 for x in cols] )
print(selected_patterns)

['$DATE|ADV|VERB+*' '$DATE|ADV|NOUN' '$DATE|ADV|NOUN+*' '$DATE|ADV|ADJ'
 '$DATE|ADV|ADJ+*']


/home/simret/miniconda3/envs/py39/lib/python3.9/site-packages/sklearn/feature_selection/_univariate_selection.py:108: RuntimeWarning: invalid value encountered in true_divide
  msb = ssbn / float(dfbn)


In [3]:
df.columns

Index(['id', 'labels', 'sentences', 'PRON', 'VERB', 'PROPN', 'PROPN|PRON',
       'PROPN|VERB', 'PROPN|NOUN', 'PROPN|ADJ',
       ...
       '$DATE|ADJ', '$DATE|ADV', '$DATE|ADV|PRON', '$DATE|ADV|PRON+*',
       '$DATE|ADV|VERB', '$DATE|ADV|VERB+*', '$DATE|ADV|NOUN',
       '$DATE|ADV|NOUN+*', '$DATE|ADV|ADJ', '$DATE|ADV|ADJ+*'],
      dtype='object', length=121)

In [14]:

df_new = df.select_dtypes(include='number')

In [15]:
df_new.columns

Index(['labels', 'PROPN', 'PROPN+NOUN', 'PROPN+*+NOUN', 'PROPN+*+[deal]',
       'PROPN+*+[deal]|NOUN', 'PROPN+[deal]', 'PROPN|NUM', 'PROPN|NUM+NOUN',
       'PROPN|NUM+*+NOUN',
       ...
       '$MONEY|[price]+*+PRON+VERB', '$MONEY|[price]+*+PRON', '$MONEY|[deal]',
       '$MONEY|[discount]', '$MONEY|[time]', '$MONEY|[time]+NOUN',
       '$MONEY|[time]+*+NOUN', '$MONEY|$ORDINAL', '$MONEY|$ORDINAL+NOUN',
       '$MONEY|$ORDINAL+*+NOUN'],
      dtype='object', length=149)

In [16]:
matrix = df_new.corr().abs()

In [17]:
mask = np.triu(np.ones_like(matrix, dtype=bool))

reduced_matrix = matrix.mask(mask)

In [18]:
reduced_matrix.iloc[:5,:5]

,labels,PROPN,PROPN+NOUN,PROPN+*+NOUN,PROPN+*+[deal]
labels,NaN,NaN,NaN,NaN,NaN
PROPN,0.067267,NaN,NaN,NaN,NaN
PROPN+NOUN,0.212351,0.371391,NaN,NaN,NaN
PROPN+*+NOUN,0.156957,0.666667,0.557086,NaN,NaN
PROPN+*+[deal],0.212351,0.371391,1.000000,0.557086,NaN


In [19]:
to_drop = [c for c in reduced_matrix.columns if any(reduced_matrix[c] ==1)]

In [20]:
to_drop

['PROPN+NOUN',
 'PROPN+*+NOUN',
 'PROPN+*+[deal]',
 'PROPN+*+[deal]|NOUN',
 'PROPN+[deal]',
 'PROPN|NUM',
 'PROPN|NUM+NOUN',
 'PROPN|NUM+*+NOUN',
 'PROPN|[price]',
 'PROPN|[deal]',
 'PROPN|[discount]',
 'PROPN|[time]',
 'PROPN|[time]+NOUN',
 'PROPN|[time]+*+NOUN',
 'PROPN|[time]+*+NOUN+ADV',
 'PROPN|$MONEY',
 'PROPN|$MONEY+NOUN',
 'PROPN|$MONEY+*+NOUN',
 'PROPN|$ORDINAL+NOUN',
 'NUM',
 'NUM+NOUN',
 'NUM+*+PRON',
 'NUM+*+PRON|PROPN',
 'NUM+*+PRON|NOUN',
 'NUM+*+PRON|$ORG',
 'NUM+*+PROPN|PRON',
 'NUM+*+PROPN',
 'NUM+*+NOUN',
 'NUM+*+$ORG|PRON',
 'NUM+*+$ORG',
 'NUM|[price]',
 'NUM|[price]+*+PRON+VERB',
 'NUM|[price]+*+PRON',
 'NUM|[price]+*+PROPN|PRON',
 'NUM|[price]+*+PROPN',
 'NUM|[deal]',
 'NUM|[discount]',
 'NUM|[time]',
 'NUM|[time]+NOUN',
 'NUM|[time]+*+NOUN',
 'NUM|$ORDINAL+NOUN',
 'NUM|$ORDINAL+*+NOUN',
 '[price]+VERB|ADV',
 '[price]+VERB|AUX',
 '[price]+ADV|AUX',
 '[price]+AUX|ADV',
 '[price]|NUM+*+PRON+VERB',
 '[price]|NUM+*+PRON',
 '[price]|[deal]',
 '[price]|[deal]+*+VERB',
 

In [21]:
len(df.columns.values)

151

In [22]:
len(to_drop)

83

In [23]:
df_reduced = df_new.drop(to_drop, axis=1)
df_reduced.shape

(30, 66)

In [24]:
df_reconstructed = df[df_reduced.columns.values]
df_reconstructed = df_reconstructed.drop("labels", axis=1)
# df_reconstructed["id"] = df['id']
# df_reconstructed["sentences"] = df["sentences"]

In [25]:
inputs = df_reconstructed.values
outs = df["labels"].values



selector = SelectKBest(f_classif, k=5)
X_new = selector.fit_transform(inputs, outs)
cols = selector.get_support(indices=True)
selected_patterns = np.take(df_reconstructed.columns.values,[x for x in cols] )
print(selected_patterns)

['[deal]|[price]' '[discount]|[price]' '[discount]|[deal]'
 '[time]|[price]' '$MONEY|[price]']


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
plt.figure(figsize=(16,12))

# Create a custom diverging palette
cmap = sns.diverging_palette(250, 15, s=75, l=40,
                             n=9, center="light", as_cmap=True)

_ = sns.heatmap(matrix, center=0, annot=True, 
                fmt='.2f', square=True, cmap=cmap)

In [115]:
from sklearn.metrics import precision_recall_fscore_support
collector = {}
labels = df['labels']
all_cols = df.columns.values[4:]
for col in all_cols:
    col_selected = df[col]
    fscore = precision_recall_fscore_support(labels, col_selected,  average="binary")[2]
    collector[col] =  fscore

In [116]:
patterns_selected = []

In [117]:
collector = {k: v for k, v in sorted(collector.items(), key=lambda item: item[1])}

In [118]:
starter = list(collector.keys())[-1]


In [119]:
corr = df.corr()

In [120]:
patterns_selected.append(starter)
to_drop = [c for c in corr.columns if corr[starter][c] >= 0.8]
df_reduced = df.drop(to_drop, axis=1)

In [121]:
import numpy as np
df_new = pd.DataFrame()
new_cols = df_reduced.columns.values[4:]
for coll in new_cols:
    
    df_new[coll] = np.logical_or(df_reduced[coll], df[starter])

/tmp/ipykernel_26584/1975192939.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_new[coll] = np.logical_or(df_reduced[coll], df[starter])


In [122]:
df_new.shape

(18, 101)

In [123]:
new_collector = {}
for col in new_cols:
    col_selected = df_reduced[col]
    fscore = precision_recall_fscore_support(labels, col_selected,  average="binary")[2]
    new_collector[col] =  fscore

In [124]:
starter, patterns_selected

('$PERCENT|[pay]', ['$PERCENT|[pay]'])

In [128]:
new_collector = {k: v for k, v in sorted(new_collector.items(), key=lambda item: item[1])}
new_collector
next_starter = list(new_collector.keys())[-1]
patterns_selected.append(next_starter)

In [127]:
next_starter

'[expensive]|[pay]+*+PRON'

In [129]:
corr = df_reduced.corr()
to_drop = [c for c in corr.columns if corr[next_starter][c] >= 0.8]
df_reduced = df_reduced.drop(to_drop, axis=1)

In [131]:
to_drop

['[pay]|[expensive]',
 '[pay]|[expensive]+*+PRON',
 '[pay]|[expensive]+*+PRON+AUX',
 '[expensive]|[pay]',
 '[expensive]|[pay]+*+PRON',
 '[expensive]|[pay]+*+PRON+AUX']

In [4]:
import torch

def train_and_report(patterns, inputs, outputs):
    #Change numpy inputs to tensors 
    outputs = torch.tensor(outputs).reshape(-1,1)
    inputs = torch.tensor(inputs)

    #train the linear layer for 100 iterations
    #100 chosen at random TODO see what a good number is for iteration

    net = torch.nn.Linear(inputs.shape[1],1, bias=False)
    sigmoid = torch.nn.Sigmoid()

    criterion = torch.nn.BCELoss()
    optimizer = torch.optim.SGD(net.parameters(), lr=0.1)

    losses = []
    net.train()
    for e in range(100):
        optimizer.zero_grad()
        o =  sigmoid.forward(net.forward(inputs.float()))
            
        loss = criterion(o, outputs.float())
            
        losses.append(loss.sum().item())
        loss.backward()
            
        optimizer.step()
    
    pred =  sigmoid.forward(net.forward(inputs.float())).detach().numpy()>0.5
    labeled_prf = precision_recall_fscore_support(outputs, pred, average="weighted")

    fscore = labeled_prf[2]
    # print(f"{patterns}, {fscore}")


    return fscore

/home/simret/miniconda3/envs/py39/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
#imports
import numpy as np
from sklearn.metrics import precision_recall_fscore_support
import pandas as pd

# define df, columns, true labels
def feature_selector(df):

    print(f"==================================Start of Feature Selection===========================================")
    labels = df['labels']
    jj = 0
    ### Controller variables
    patterns_selected = []
    highest_fscore = "0.0"
    df_subset = pd.DataFrame()
    remaining_cols = df.columns.values[4:]



    outputs = df["labels"].values
    while len(patterns_selected)<10 and len(remaining_cols)>0:
        jj += 1
        print(f"Starting iteration {jj} {len(remaining_cols)}")
        #first calculate the fscore
        collector = {}
        local_max_fscore = "0.0"
        for col in remaining_cols:
            col_selected = df[col].astype('int64')
            current_patterns = patterns_selected+[col]
            current_df = pd.concat([df_subset, col_selected], axis=1)
            inputs = current_df.values
            
            # fscore = precision_recall_fscore_support(labels, col_selected,  average="binary")[2]
            fscore = train_and_report(current_patterns, inputs, outputs)
            
                
            exists = str(fscore) in collector
            if(exists):
                collector[str(fscore)].append(col)
                
            else:
                collector[str(fscore)] = [col]
        #sort and get a pattern with high fscore
        selected_starter_pattern = list(collector.values())[-1]
        collector = {k: v for k, v in sorted(collector.items(), key=lambda item: item[0])}
        current_fscore = list(collector.keys())[-1]

        if(current_fscore>highest_fscore):
            highest_fscore = current_fscore
        else:
            # print(f"{highest_fscore} {current_fscore}, {selected_starter_pattern}")
            break
        selected_starter_pattern = list(collector.values())[-1]
        # print(selected_starter_pattern)

        #Group the correlated ones and pick the shortest
        rowss = df[selected_starter_pattern]
        correlation = rowss.corr()
        correlation.loc[:,:] =  np.tril(correlation, k=-1)
        cor = correlation.stack()
        ones = cor[cor >=0.8].reset_index().loc[:,['level_0','level_1']]
        ones = ones.query('level_0 not in level_1')
        grps = list(ones.groupby('level_0').groups.keys())
        colls = []
        for i in grps:
            groups = ones[ones["level_0"]==i].values
            set_maker = []
            for patterns in groups:
                set_maker += patterns.tolist()
            colls.append(sorted(set_maker, key=len)[0])
            
        for selected_starter_pattern in colls:
            patterns_selected.append(selected_starter_pattern)
            df_subset[selected_starter_pattern] = df[selected_starter_pattern].astype('int64')
            try:
                selected_starter_series = df[selected_starter_pattern][0]
                
                corr = df.corr()
                to_drop = [c for c in corr.columns if corr[selected_starter_pattern][c] >= 0.8] #0.8 chosen at random
                df = df.drop(to_drop, axis=1)

                #create a new df with combination of current one
                remaining_cols = df.columns.values[4:]
                for collumn in remaining_cols:
                    df[collumn] = np.logical_or(df[collumn], selected_starter_series)
            except:
                print("We already removed ", selected_starter_pattern)
            for coll in remaining_cols:
                df[coll] = np.logical_or(df[coll], selected_starter_series)
        
        print(f"Finishing iteration {jj} {len(remaining_cols)}, --- {patterns_selected}, {highest_fscore}")
    
    print(f"---------------------------Summary---------------------------")
    print(f"Patterns {patterns_selected}")
    print(f"Positive examples \n{df[df['labels']==1]['sentences'].values}")
    print(f"Negative examples \n{df[df['labels']==0]['sentences'].values}")

    print(f"==================================End of Feature Selection===========================================")
    return patterns_selected




In [14]:
import warnings
warnings.filterwarnings('ignore')

df = pd.read_csv("res.csv")
print(df.shape)

features = feature_selector(df)

(5, 121)
==================================Start of Feature Selection===========================================
Starting iteration 1 117
Finishing iteration 1 117, --- [], 1.0
Starting iteration 2 117
---------------------------Summary---------------------------
Patterns []
Positive examples 
['Had a really awkward time ordering a beer because nothing on the menu was stocked . '
 'I really enjoyed ( non - sarcastically ) having an overpriced drink here because it reminded me of New York . '
 'We got here just before the peak lunch crowd on a Saturday morning , and still had to wait 30 minutes , which gave us enough time to find parking down the street . '
 "Hence the food is way too pricey for what 's offered "
 'I can not believe someone was not satisfied with the quality of the chicken at this location ! ']
Negative examples 
[]
==================================End of Feature Selection===========================================


# New Approach

In [31]:
def feature_selector_2(df, k):
    patterns = []
    # inputs = df.iloc[:,3:].values
    outs = df["labels"].values

    for i in range(k):

        inputs = df.iloc[:,3:].values
        

        selector = SelectKBest(f_classif, k=1)
        X_new = selector.fit_transform(inputs, outs)
        cols = selector.get_support(indices=True)
        selected_patterns = np.take(df.columns.values,[x+3 for x in cols] )
        #get rid of all features correlated 
        corr = df.corr()
        corr.head()
        # print(corr)
        # return True
        
        to_drop = [c for c in corr.columns if corr[selected_patterns[0]][c] >= 0.5 or pd.isnull(corr[selected_patterns[0]][c])] #0.8 chosen at random
        # print("We will drop ", to_drop)

        df = df.drop(to_drop, axis=1)
        print(f'picked {selected_patterns[0]}')
        
        # print(df.shape[1])

        
        patterns.append(selected_patterns[0])
        if(df.shape[1]<=4):
            break
    return patterns, corr

In [32]:
c = feature_selector_2(df, 5)

picked $DATE|ADV|ADJ+*


In [35]:
c[1]["$DATE|ADV|ADJ+*"]

labels             NaN
PRON               NaN
VERB               NaN
PROPN              NaN
PROPN|PRON         NaN
                    ..
$DATE|ADV|VERB+*   NaN
$DATE|ADV|NOUN     NaN
$DATE|ADV|NOUN+*   NaN
$DATE|ADV|ADJ      NaN
$DATE|ADV|ADJ+*    NaN
Name: $DATE|ADV|ADJ+*, Length: 119, dtype: float64

In [24]:
corr.to_csv("corr.csv")

In [30]:
['[price]|[discount]' '[price]|$MONEY' '[discount]|[price]'
 '[time]|[price]' '$MONEY|[price]']

['[price]|[discount][price]|$MONEY[discount]|[price][time]|[price]$MONEY|[price]']

In [8]:
corr

,labels,PROPN,PROPN+NOUN,PROPN+*+NOUN,PROPN+*+[deal],PROPN+*+[deal]|NOUN,PROPN+[deal],PROPN|NUM,PROPN|NUM+NOUN,PROPN|NUM+*+NOUN,...,$MONEY|[price]+*+PRON+VERB,$MONEY|[price]+*+PRON,$MONEY|[deal],$MONEY|[discount],$MONEY|[time],$MONEY|[time]+NOUN,$MONEY|[time]+*+NOUN,$MONEY|$ORDINAL,$MONEY|$ORDINAL+NOUN,$MONEY|$ORDINAL+*+NOUN
labels,1.000000,0.067267,0.212351,0.156957,0.212351,0.156957,0.212351,0.161468,-0.145114,-0.030083,...,0.381181,0.381181,5.114083e-01,5.114083e-01,5.114083e-01,0.305625,0.381181,0.330911,0.305625,0.305625
PROPN,0.067267,1.000000,0.371391,0.666667,0.371391,0.666667,0.371391,0.763763,0.539319,0.670820,...,-0.166667,-0.166667,9.930137e-17,7.447602e-17,4.965068e-17,0.200446,0.111111,0.223607,0.200446,0.200446
PROPN+NOUN,0.212351,0.371391,1.000000,0.557086,1.000000,0.557086,1.000000,0.283654,0.473432,0.415227,...,-0.061898,-0.061898,4.152274e-01,4.152274e-01,4.152274e-01,0.694808,0.557086,0.415227,0.694808,0.694808
PROPN+*+NOUN,0.156957,0.666667,0.557086,1.000000,0.557086,1.000000,0.557086,0.509175,0.522976,0.745356,...,-0.111111,-0.111111,1.490712e-01,1.490712e-01,1.490712e-01,0.356348,0.259259,0.447214,0.356348,0.356348
PROPN+*+[deal],0.212351,0.371391,1.000000,0.557086,1.000000,0.557086,1.000000,0.283654,0.473432,0.415227,...,-0.061898,-0.061898,4.152274e-01,4.152274e-01,4.152274e-01,0.694808,0.557086,0.415227,0.694808,0.694808
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
$MONEY|[time]+NOUN,0.305625,0.200446,0.694808,0.356348,0.694808,0.356348,0.694808,0.116642,0.288278,0.239046,...,-0.089087,-0.089087,2.390457e-01,5.976143e-01,5.976143e-01,1.000000,0.801784,0.597614,1.000000,1.000000
$MONEY|[time]+*+NOUN,0.381181,0.111111,0.557086,0.259259,0.557086,0.259259,0.557086,0.024246,0.196116,0.149071,...,-0.111111,-0.111111,1.490712e-01,4.472136e-01,7.453560e-01,0.801784,1.000000,0.447214,0.801784,0.801784
$MONEY|$ORDINAL,0.330911,0.223607,0.415227,0.447214,0.415227,0.447214,0.415227,0.487950,0.350823,0.280000,...,0.447214,0.447214,5.200000e-01,7.600000e-01,7.600000e-01,0.597614,0.447214,1.000000,0.597614,0.597614
$MONEY|$ORDINAL+NOUN,0.305625,0.200446,0.694808,0.356348,0.694808,0.356348,0.694808,0.116642,0.288278,0.239046,...,-0.089087,-0.089087,2.390457e-01,5.976143e-01,5.976143e-01,1.000000,0.801784,0.597614,1.000000,1.000000
